In [1]:
# Dependencies
import numpy as np

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [2]:
engine = create_engine("sqlite:///hawaii.sqlite")

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [3]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [4]:
import datetime as dt
import pandas as pd

## D1: Determine the Summary Statistics for June

In [5]:
# 1. Import the sqlalchemy extract function.
from sqlalchemy import extract

# 2. Write a query that filters the Measurement table to retrieve the temperatures for the month of June. 
june_start = dt.date(2016,6,1)
june_end = dt.date(2016,6,30)

temps_june = []
temps_june = session.query(Measurement.date, Measurement.tobs).\
filter((Measurement.date >= june_start) & (Measurement.date <= june_end))

In [6]:
#  3. Convert the June temperatures to a list.
temps_june = list(temps_june)

In [7]:
# 4. Create a DataFrame from the list of temperatures for the month of June. 
df = pd.DataFrame(temps_june,columns=['date','temps'])
df.set_index(df['date'], inplace=True)
df.sort_index()

,date,temps
date,,
2016-06-01,2016-06-01,76.0
2016-06-01,2016-06-01,70.0
2016-06-01,2016-06-01,70.0
2016-06-01,2016-06-01,69.0
2016-06-01,2016-06-01,75.0
...,...,...
2016-06-30,2016-06-30,78.0
2016-06-30,2016-06-30,79.0
2016-06-30,2016-06-30,77.0


In [8]:
# 5. Calculate and print out the summary statistics for the June temperature DataFrame.
df.describe()

,temps
count,194.000000
mean,75.175258
std,3.421996
min,64.000000
25%,73.000000
50%,75.500000
75%,78.000000
max,81.000000


## D2: Determine the Summary Statistics for December

In [9]:
# 6. Write a query that filters the Measurement table to retrieve the temperatures for the month of December.
dec_start = dt.date(2016,12,1)
dec_end = dt.date(2016,12,31)
dec_temps = session.query(Measurement.date, Measurement.tobs).\
filter((Measurement.date >= dec_start) & (Measurement.date <= dec_end))

In [10]:
# The sample data's max data only goes up until August 2017, so the earliest December month is from 2016. 
session.query(func.max(Measurement.date)).all()

[('2017-08-23',)]

In [11]:
# 7. Convert the December temperatures to a list.
dec_temps = list(dec_temps)
len(dec_temps)

200

In [12]:
# 8. Create a DataFrame from the list of temperatures for the month of December. 
df_dec = pd.DataFrame(dec_temps, columns=['date','temps'])
df_dec.set_index(df_dec['date'],inplace=True)
df_dec.sort_index()

,date,temps
date,,
2016-12-01,2016-12-01,75.0
2016-12-01,2016-12-01,76.0
2016-12-01,2016-12-01,75.0
2016-12-01,2016-12-01,74.0
2016-12-01,2016-12-01,73.0
...,...,...
2016-12-31,2016-12-31,66.0
2016-12-31,2016-12-31,71.0
2016-12-31,2016-12-31,72.0


In [13]:
# 9. Calculate and print out the summary statistics for the Decemeber temperature DataFrame.
df_dec.describe()

,temps
count,200.000000
mean,71.130000
std,3.419292
min,60.000000
25%,69.000000
50%,71.000000
75%,74.000000
max,78.000000
